In [69]:
# !pip install linearmodels

In [62]:
import pandas as pd
import numpy as np
import calendar as cd
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [63]:
# load in current data
df = pd.read_excel('Durham Zillow Filled.xlsx', sheet_name = 'Repeated Sales Homes')

In [64]:
# make sure datatypes are correct
df.dtypes

OBJECT_ID           int64
PARCEL_ID           int64
Site Address       object
Data Source        object
Site City          object
Site State         object
Site ZIP            int64
Activity           object
X                 float64
Y                 float64
PIN                object
Year of Issue       int64
Month of Issue      int64
Cost                int64
Permit Value      float64
Sale Value         object
dtype: object

In [65]:
# create dummies for year and month
df_dum = pd.get_dummies(df, columns=['Year of Issue', 'Month of Issue'])

# years
df_dum.iloc[:,14:32]

# months
df_dum.iloc[:,32:]

,Month of Issue_1,Month of Issue_2,Month of Issue_3,Month of Issue_4,Month of Issue_5,Month of Issue_6,Month of Issue_7,Month of Issue_8,Month of Issue_9,Month of Issue_10,Month of Issue_11,Month of Issue_12
0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
38432,0,0,0,0,0,0,0,0,0,1,0,0
38433,0,0,0,0,0,0,0,0,0,0,0,1
38434,0,0,0,0,1,0,0,0,0,0,0,0
38435,0,0,0,0,0,0,0,0,0,0,0,1


In [66]:
# create empty data frame to hold interaction terms
df2 = pd.DataFrame(data={})

df2

""


In [67]:
# get the columns from the original table except for the ones that we are about to dummy
for col in df_dum.iloc[:,:14]:
    df2[col] = df_dum[col]
    
df2

,OBJECT_ID,PARCEL_ID,Site Address,Data Source,Site City,Site State,Site ZIP,Activity,X,Y,PIN,Cost,Permit Value,Sale Value
0,4,100016,901 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912429,36.011606,0822-15-53-8193,0,0.0,53000
1,4,100016,901 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912429,36.011606,0822-15-53-8193,0,15000.0,53000
2,8,100030,921 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912442,36.012984,0822-15-53-8693,0,0.0,219000
3,8,100030,921 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912442,36.012984,0822-15-53-8693,0,65000.0,320000
4,8,100030,921 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912442,36.012984,0822-15-53-8693,0,65000.0,340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38432,39753,313151,9919 Wilkins Rd,Zillow/Real Sale,Bahama,NC,27503,S,-78.866007,36.182639,313151,0,0.0,167960
38433,39753,313151,9919 Wilkins Rd,Zillow/Real Sale,Bahama,NC,27503,S,-78.866007,36.182639,313151,0,0.0,156000
38434,39754,313152,9939 Quail Roost Rd,Zillow/Real Sale,Bahama,NC,27503,S,-78.890454,36.183173,313152,0,0.0,490000
38435,39754,313152,9939 Quail Roost Rd,Zillow/Real Sale,Bahama,NC,27503,S,-78.890454,36.183173,313152,0,0.0,540000


In [68]:
# list of years 2001-2020
years = range(2001,2021)

# get interaction terms by multiplying
for year in years:
    # there is no 2002 or 2004 in the records, for some reason
    if year == 2002 or year == 2004:
        continue
    
    for j in range(1,13):
        # skip the first interaction term, since it is redundant
        # (consider all 0s on other interaction terms)
        if (year == 2001 and j == 1):
            continue
        
        # multiply columns to get interactions
        df2[cd.month_name[j] + " " + str(year)] = df_dum['Year of Issue_'+str(year)]*df_dum['Month of Issue_'+str(j)]

df2
        

,OBJECT_ID,PARCEL_ID,Site Address,Data Source,Site City,Site State,Site ZIP,Activity,X,Y,...,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,October 2020,November 2020,December 2020
0,4,100016,901 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912429,36.011606,...,0,0,0,0,0,0,0,0,0,0
1,4,100016,901 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912429,36.011606,...,0,0,0,0,0,0,0,0,0,0
2,8,100030,921 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912442,36.012984,...,0,0,0,0,0,0,0,0,0,0
3,8,100030,921 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912442,36.012984,...,0,0,0,0,0,0,0,0,0,0
4,8,100030,921 N Buchanan Blvd,Zillow/Real Sale,Durham,NC,27701,S,-78.912442,36.012984,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38432,39753,313151,9919 Wilkins Rd,Zillow/Real Sale,Bahama,NC,27503,S,-78.866007,36.182639,...,0,0,0,0,0,0,0,0,0,0
38433,39753,313151,9919 Wilkins Rd,Zillow/Real Sale,Bahama,NC,27503,S,-78.866007,36.182639,...,0,0,0,0,0,0,0,0,0,0
38434,39754,313152,9939 Quail Roost Rd,Zillow/Real Sale,Bahama,NC,27503,S,-78.890454,36.183173,...,0,0,0,0,0,0,0,0,0,0
38435,39754,313152,9939 Quail Roost Rd,Zillow/Real Sale,Bahama,NC,27503,S,-78.890454,36.183173,...,0,0,0,0,0,0,0,0,0,0
